In [ ]:
!yt-dlp --rm-cache-dir

In [ ]:
# ==============================================================================
# BƯỚC 1: CÀI ĐẶT & MOUNT DRIVE
# ==============================================================================
import os
import csv
import time
import sys
from google.colab import drive

# 1. Mount Drive
if not os.path.exists('/content/drive'):
    print("📂 Đang kết nối Google Drive...")
    drive.mount('/content/drive')

# 2. Cài thư viện
print("⬇️ Đang cài đặt thư viện...")
!apt-get install ffmpeg -y > /dev/null
!pip install yt-dlp faster-whisper > /dev/null 2>&1

import yt_dlp
from faster_whisper import WhisperModel

In [ ]:
!yt-dlp --rm-cache-dir

In [ ]:
# ==============================================================================
# BƯỚC 2: CẤU HÌNH FILE
# ==============================================================================
PROJECT_FOLDER = "/content/drive/MyDrive/TikTok_Project"

# Tên file đầu vào (Bạn đổi tên file của bạn thành vtv24_real.csv hoặc sửa dòng này)
INPUT_CSV = os.path.join(PROJECT_FOLDER, "tiktok_videos_all_keywords_real.csv")
COOKIES_FILE = os.path.join(PROJECT_FOLDER, "cookies.txt")

# File kết quả
OUTPUT_CSV = os.path.join(PROJECT_FOLDER, "tiktok_videos_all_keywords_real_done.csv")

if not os.path.exists(INPUT_CSV):
    print(f"❌ LỖI: Không tìm thấy file '{INPUT_CSV}'")
    sys.exit()

# ==============================================================================
# BƯỚC 3: LOAD MODEL GPU
# ==============================================================================
print("🚀 Đang load model Whisper Large-v3...")
model = WhisperModel("large-v3", device="cuda", compute_type="float16")
print("✅ Model sẵn sàng!")

# ==============================================================================
# BƯỚC 4: HÀM XỬ LÝ (Download & Transcribe)
# ==============================================================================
def download_audio(video_url, video_id):
    if not video_url: return None
    temp_output = f"/content/temp_{video_id}"

    # Lấy ID thật từ URL (VD: .../video/7569111204208069895 -> 7569111204208069895)
    try:
        real_id_from_url = video_url.split('/video/')[1].split('?')[0]
    except:
        real_id_from_url = ""

    ydl_opts = {
        'outtmpl': f'{temp_output}.%(ext)s',
        'format': 'bestaudio/best',
        'noplaylist': True,
        'quiet': True,
        'no_warnings': True,
        'ignoreerrors': True,
        'http_headers': {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
            'Referer': 'https://www.tiktok.com/',
        },
        'cookiefile': COOKIES_FILE if os.path.exists(COOKIES_FILE) else None,

        # --- CHỐNG REDIRECT (QUAN TRỌNG NHẤT) ---
        # Chỉ tải nếu ID khớp. Nếu bị chuyển sang video khác -> Bỏ qua ngay
        'match_filter': yt_dlp.utils.match_filter_func("id")
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            # 1. Thử lấy thông tin JSON trước (nhẹ hơn tải file)
            info = ydl.extract_info(video_url, download=False)

            if not info:
                return None

            # 2. Kiểm tra chéo ID
            detected_id = info.get('id')
            if real_id_from_url and detected_id != real_id_from_url:
                print(f"   🚨 CẢNH BÁO: TikTok đang lừa bạn! (Link gốc: {real_id_from_url} -> Bị lái sang: {detected_id})")
                return None # Hủy kèo, không tải

            # 3. Nếu khớp ID thì mới tải
            ydl.download([video_url])

        for ext in ['.m4a', '.mp3', '.webm', '.mp4']:
            path = f"{temp_output}{ext}"
            if os.path.exists(path): return path
        return None
    except Exception as e:
        # print(f"Lỗi: {e}")
        return None

def transcribe_audio(audio_path):
    try:
        segments, info = model.transcribe(audio_path, language="vi", beam_size=5)
        return " ".join([s.text for s in segments]).strip()
    except: return ""

# ==============================================================================
# BƯỚC 5: MAIN LOOP (Xử lý CSV thông minh)
# ==============================================================================
def main():
    print(f"📂 Input: {INPUT_CSV}")
    print(f"💾 Output: {OUTPUT_CSV}")

    # 1. Đọc Header của file cũ để lấy danh sách cột
    input_rows = []
    with open(INPUT_CSV, 'r', encoding='utf-8') as f_in:
        reader = csv.DictReader(f_in)
        fieldnames = reader.fieldnames # Lấy danh sách cột cũ
        # Lưu lại toàn bộ data vào list để xử lý (hoặc xử lý stream nếu file quá lớn)
        # Ở đây file vài nghìn dòng thì load vào RAM cho an toàn
        input_rows = list(reader)

    # 2. Thêm 2 cột mới vào danh sách Header
    if 'text' not in fieldnames: fieldnames.append('text')
    if 'category' not in fieldnames: fieldnames.append('category')

    # 3. Kiểm tra resume (nếu đang chạy dở)
    processed_urls = set()
    file_exists = os.path.exists(OUTPUT_CSV)
    if file_exists:
        with open(OUTPUT_CSV, 'r', encoding='utf-8') as f_out:
            reader = csv.DictReader(f_out)
            for row in reader:
                processed_urls.add(row['url'])
        print(f"🔄 Đã xử lý trước đó: {len(processed_urls)} dòng.")

    # 4. Mở file Output để ghi (Append mode)
    mode = 'a' if file_exists else 'w'
    with open(OUTPUT_CSV, mode, newline='', encoding='utf-8') as f_out:
        writer = csv.DictWriter(f_out, fieldnames=fieldnames)

        # Nếu file mới tinh thì ghi Header
        if mode == 'w':
            writer.writeheader()

        # 5. Duyệt qua từng dòng của file gốc
        count = 0
        total = len(input_rows)

        for i, row in enumerate(input_rows, 1):
            url = row.get('url', '')

            # Nếu URL này đã làm rồi thì bỏ qua
            if url in processed_urls:
                continue

            print(f"[{i}/{total}] ⏳ {url}")

            # --- Xử lý chính ---
            text_result = ""
            audio_path = None

            try:
                # Tải audio
                unique_id = f"real_{i}_{int(time.time())}"
                audio_path = download_audio(url, unique_id)

                if audio_path:
                    # Chuyển đổi sang text
                    text_result = transcribe_audio(audio_path)
                    print(f"   ✅ Text: {text_result[:50]}...")
                else:
                    print("   ❌ Không tải được video")
            except Exception as e:
                print(f"   ⚠️ Lỗi: {e}")
            finally:
                # Dọn dẹp file tạm
                if audio_path and os.path.exists(audio_path):
                    os.remove(audio_path)

            # --- Cập nhật dữ liệu vào dòng hiện tại ---
            row['text'] = text_result      # Gán text
            row['category'] = 'real'       # Gán nhãn real

            # Ghi dòng này vào file output
            writer.writerow(row)

            # Flush để lưu ngay lập tức
            f_out.flush()
            count += 1

    print(f"\n🎉 HOÀN THÀNH! Đã xử lý {count} dòng mới.")
    print(f"File lưu tại: {OUTPUT_CSV}")

if __name__ == "__main__":
    main()